In [ ]:
import jigsolver
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img_folder = 'img/'
img = plt.imread(img_folder + "eiffel.jpg")
print(img.shape)
plt.imshow(img)
plt.show()

In [ ]:
eiffel_puzzle = jigsolver.Puzzle(img)
eiffel_puzzle.create()

In [ ]:
eiffel_puzzle.plot()

In [ ]:
jigsolver.random_solver(eiffel_puzzle)